# AML Assignment-2

| Name                | ID         |
|---------------------|------------|
| Ananya Sinha        | MDS202307  |
| Divyanshi Kumari    | MDS202322  |
| Rohit Roy           | MDS202340  |

# Question 1

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import SimpleRNN, LSTM, Activation, Dense, Dropout, Embedding

2024-10-26 13:10:37.319284: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-26 13:10:37.334222: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-26 13:10:37.338777: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-26 13:10:37.349834: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-26 13:10:38.129369: W tensorflow/compiler/tf2

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

### Reading and preparing the data

In [3]:
sms_df = pd.read_csv('data/spam.csv', encoding='ISO-8859-1')
sms_df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
sms_df.dropna(how="any", inplace=True, axis=1)
sms_df.columns = ['label', 'message']
sms_df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
# Preprocessing
sms_df['label'] = sms_df['label'].map({'ham': 0, 'spam': 1})
messages = sms_df['message'].values
labels = sms_df['label'].values

# Tokenization
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(messages)
sequences = tokenizer.texts_to_sequences(messages)

# Padding sequences
maxlen = 150  # Define max length of messages
X = pad_sequences(sequences, maxlen=maxlen)
y = labels

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Defining and training the models

In [6]:
# Define common parameters for both models
vocab_size = 5000
embedding_dim = 50
input_length = maxlen

rnn_model = Sequential()
rnn_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=input_length))
rnn_model.add(SimpleRNN(32))  # RNN layer
rnn_model.add(Dense(256, name='FC1'))
rnn_model.add(Activation('relu'))
rnn_model.add(Dropout(0.25))
rnn_model.add(Dense(1, activation='sigmoid'))  # Output layer

rnn_model.build(input_shape=(None, input_length))
rnn_model.summary()
rnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
rnn_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=1)


lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=input_length))
lstm_model.add(LSTM(32))  # LSTM layer
lstm_model.add(Dense(1, activation='sigmoid'))  # Output layer

lstm_model.build(input_shape=(None, input_length))
lstm_model.summary()
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=1)

# Evaluate models
rnn_test_loss, rnn_test_accuracy = rnn_model.evaluate(X_test, y_test, verbose=0)
lstm_test_loss, lstm_test_accuracy = lstm_model.evaluate(X_test, y_test, verbose=0)

print(f'RNN Test Accuracy: {rnn_test_accuracy:.4f}')
print(f'LSTM Test Accuracy: {lstm_test_accuracy:.4f}')

/home/rohitroy/miniconda3/envs/assigns/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2024-10-26 13:10:40.091242: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9263 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:05:00.0, compute capability: 7.5


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 150, 50)        │       250,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 32)             │         2,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ FC1 (Dense)                     │ (None, 256)            │         8,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261,361 (1020.94 KB)

 Trainable params: 261,361 (1020.94 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 150, 50)        │       250,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 32)             │        10,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 260,657 (1018.19 KB)

 Trainable params: 260,657 (1018.19 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.8899 - loss: 0.3406 - val_accuracy: 0.9794 - val_loss: 0.0770
Epoch 2/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9886 - loss: 0.0386 - val_accuracy: 0.9839 - val_loss: 0.0537
Epoch 3/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9965 - loss: 0.0158 - val_accuracy: 0.9839 - val_loss: 0.0515
Epoch 4/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9987 - loss: 0.0096 - val_accuracy: 0.9821 - val_loss: 0.0722
Epoch 5/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9961 - loss: 0.0120 - val_accuracy: 0.9839 - val_loss: 0.0572
Epoch 6/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9979 - loss: 0.0056 - val_accuracy: 0.9848 - val_loss: 0.0589
Epoch 7/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9981 - loss: 0.0040 - val_accuracy: 0.9641 - val_loss: 0.1083
Epoch 8/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9967 - loss: 0.0105 - val_accu

### Accuracies :

RNN : 98.39% with 261,361 paramters

LSTM : 98.48% with 260,657 parameters

### Example predictions

In [7]:
# Example predictions on test set
example_indices = np.random.choice(len(X_test), size=3, replace=False)
example_messages = [X_test[i] for i in example_indices]  # Use X_test, which is tokenized and padded
original_messages = [sms_df['message'].iloc[i] for i in example_indices]  # Retrieve original SMS texts
example_labels = [y_test[i] for i in example_indices]

# Convert example_messages back to an array if needed
example_messages = np.array(example_messages)

# RNN Predictions
rnn_predictions = (rnn_model.predict(example_messages) > 0.5).astype('int32')

# LSTM Predictions
lstm_predictions = (lstm_model.predict(example_messages) > 0.5).astype('int32')

# Print some example messages, predictions, and original text
print("\n--- Example Predictions---")
for i, idx in enumerate(example_indices):
    print(f"Original Message: {original_messages[i]}")
    print(f"Actual Label: {'ham' if example_labels[i] == 0 else 'spam'}")
    print(f"RNN Prediction: {'ham' if rnn_predictions[i] == 0 else 'spam'}")
    print(f"LSTM Prediction: {'ham' if lstm_predictions[i] == 0 else 'spam'}\n")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 414ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step

--- Example Predictions---
Original Message: I hope you that's the result of being consistently intelligent and kind. Start asking him about practicum links and keep your ears open and all the best. ttyl
Actual Label: spam
RNN Prediction: spam
LSTM Prediction: spam

Original Message: No my blankets are sufficient, thx
Actual Label: ham
RNN Prediction: ham
LSTM Prediction: ham

Original Message: We still on for tonight?
Actual Label: ham
RNN Prediction: ham
LSTM Prediction: ham



In [8]:
# RNN Predictions
rnn_predictions = (rnn_model.predict(X_test) > 0.5).astype('int32')

# LSTM Predictions
lstm_predictions = (lstm_model.predict(X_test) > 0.5).astype('int32')

# Print only cases where RNN and LSTM predictions differ
c = 0
print("\n--- Differing Predictions in Entire Test Set ---")
for i in range(len(X_test)):
    
    if rnn_predictions[i] != lstm_predictions[i]:
        c +=1
        original_message = sms_df['message'].iloc[i]  # Retrieve original SMS text
        actual_label = 'ham' if y_test[i] == 0 else 'spam'
        rnn_pred = 'ham' if rnn_predictions[i] == 0 else 'spam'
        lstm_pred = 'ham' if lstm_predictions[i] == 0 else 'spam'
        
        print(f"Original Message: {original_message}")
        print(f"Actual Label: {actual_label}")
        print(f"RNN Prediction: {rnn_pred}")
        print(f"LSTM Prediction: {lstm_pred}\n")
    if c==4:
        break

35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

--- Differing Predictions in Entire Test Set ---
Original Message: Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
Actual Label: ham
RNN Prediction: ham
LSTM Prediction: spam

Original Message: You are always putting your business out there. You put pictures of your ass on facebook. You are one of the most open people i've ever met. Why would i think a picture of your room would hurt you, make you feel violated.
Actual Label: spam
RNN Prediction: ham
LSTM Prediction: spam

Original Message: Hi Princess! Thank you for the pics. You are very pretty. How are you?
Actual Label: spam
RNN Prediction: ham
LSTM Prediction: spam

Original Message: Wait, do you know if wesleys in town? I bet she does hella drugs!
Actual Label: spam
RNN Prediction: spam
LSTM Prediction: ham



### Classification report

In [9]:
from sklearn.metrics import classification_report
print("RNN Classification Report : ")
print(classification_report(y_test, rnn_predictions))
print("LSTM Classification Report : ")
print(classification_report(y_test, lstm_predictions))

RNN Classification Report : 
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       965
           1       0.98      0.90      0.94       150

    accuracy                           0.98      1115
   macro avg       0.98      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115

LSTM Classification Report : 
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       965
           1       0.98      0.91      0.94       150

    accuracy                           0.98      1115
   macro avg       0.98      0.95      0.97      1115
weighted avg       0.98      0.98      0.98      1115

